In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns',None)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

import matplotlib.pyplot as plt 

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns 
from tqdm import tqdm
tqdm.pandas()

import datetime
import ast
import math
import gc

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=7, progress_bar=True)

from sklearn.preprocessing import MinMaxScaler

def say(comment):
  os.system(f'say "{comment}"')

# warming 제거
import warnings
warnings.filterwarnings('ignore')
from branca.colormap import linear

# notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것
%matplotlib inline       

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
s2p = pd.read_pickle('/Users/beomso0/Documents/GH/Hyundai/RawData2/s2p/0211_1353_skill2pgm_1차완성_ver3.pkl')

In [5]:
s2p['쇼호스트명'].apply(lambda x: len(x) if x is not None else 0).max()

5

In [6]:
s2p.head()

,방송편성번호,인입콜,year,month,day,hour,PGM명,상품구분,쇼호스트명,날씨,기온,강수량,방송시작일시,weekday,duration,sorted_판매상품명,sorted_상품대분류명,sorted_상품중분류명,sorted_상품소분류명,sorted_상품세분류명,sorted_브랜드명,comb_판매상품명,comb_상품대분류명,comb_상품중분류명,comb_상품소분류명,comb_상품세분류명,comb_브랜드명,상품판매가,판매가평균,midcat_order_score,midcat_etc_score,midcat_order_score_mean,midcat_etc_score_mean,midcat_order_score_max,midcat_etc_score_max,prod_num,한정표현구분,한정표현_num,주말/공휴일,sh_num,sh_grade,sh_grade_num,시차변수,소비지수,month_sin,month_cos,hour_sin,hour_cos,day_sin,day_cos,approx_in_show_call,approx_out_show_call
0,202001015001,33,2020.0,1,1,6,(D19_P)소담 진공쌀통(김홍근10_15),DATA,"[엄지혜, 이경은]",구름 많음,-4.0,0.0,2020-01-01 06:00:00,수,40,[소담 진공쌀통],[주방],[보관용품],[쌀독],[쌀통],[소담],[소담 진공쌀통],[주방],[보관용품],[쌀독],[쌀통],[소담],[99000],99000.0,[2],[2],2.0,2.0,2,2,1,[없음],0,0,2.0,D,1.0,NaN,104.8,0.5,0.866025,1.000000,6.123234e-17,0.201299,0.97953,19.208955,13.791045
1,202001015002,80,2020.0,1,1,6,(D19_N)코지마 3D 목어깨 마사지기(박정우09_23),DATA,"[신성애, 장민철]",구름 많음,-4.0,0.0,2020-01-01 06:40:00,수,60,[코지마 3D 목어깨 마사지기 트위스터 CMN-100WL],[생활/건강],[건강],[안마/마사지기],[마사지기],[코지마],[코지마 3D 목어깨 마사지기 트위스터 CMN-100WL],[생활/건강],[건강],[안마/마사지기],[마사지기],[코지마],[99000],99000.0,[3],[3],3.0,3.0,3,3,1,[없음],0,1,2.0,D,1.0,NaN,104.8,0.5,0.866025,1.000000,6.123234e-17,0.201299,0.97953,53.698630,26.301370
2,202001015003,203,2020.0,1,1,7,(D19_Y)헤드골프 하이브리드 패딩수트(주재광11_28),DATA,"[심용수, 유지수]",구름 많음,-4.0,0.0,2020-01-01 07:40:00,수,60,"[헤드골프 남성 하이브리드 패딩수트 상하의 2종, 헤드골프 여성 하이브리드 패딩수트...","[스포츠/레져, 스포츠/레져]","[골프용품, 골프용품]","[골프웨어, 골프웨어]","[상하의세트(남성), 상하의세트(여성)]","[헤드, 헤드]","[헤드골프 남성 하이브리드 패딩수트 상하의 2종, 헤드골프 여성 하이브리드 패딩수트...","[스포츠/레져, 스포츠/레져]","[골프용품, 골프용품]","[골프웨어, 골프웨어]","[상하의세트(남성), 상하의세트(여성)]","[헤드, 헤드]","[99000, 99000]",99000.0,"[2, 2]","[4, 4]",2.0,4.0,2,4,2,[없음],0,1,2.0,C,2.0,NaN,104.8,0.5,0.866025,0.965926,-2.588190e-01,0.201299,0.97953,150.409326,52.590674
3,202001015004,188,2020.0,1,1,8,(D18_B)진휘 때필링(임흥준10_18),DATA,[이찬석],흐림,0.0,0.0,2020-01-01 08:40:00,수,60,"[진휘 때필링 1 세트, 진휘 때필링 세트]","[이미용, 이미용]","[이미용품, 이미용품]","[바디용품, 바디용품]","[필링, 필링]","[진휘, 진휘]","[진휘 때필링 세트, 진휘 때필링 1 세트]","[이미용, 이미용]","[이미용품, 이미용품]","[바디용품, 바디용품]","[필링, 필링]","[진휘, 진휘]","[49900, 39000]",44450.0,"[2, 2]","[2, 2]",2.0,2.0,2,2,2,[없음],0,1,1.0,S,5.0,NaN,104.8,0.5,0.866025,0.866025,-5.000000e-01,0.201299,0.97953,51.700000,136.300000
4,202001015005,72,2020.0,1,1,9,(D19_N)아이스크림 홈런(양선10_18),DATA,[신진아],흐림,0.0,0.0,2020-01-01 09:40:00,수,60,[[렌탈]아이스크림 홈런],[서비스],[회원권/이용권],[렌탈(일반)],[학습기기],[아이스크림홈런],[[렌탈]아이스크림 홈런],[서비스],[회원권/이용권],[렌탈(일반)],[학습기기],[아이스크림홈런],[2332500],2332500.0,[3],[3],3.0,3.0,3,3,1,[없음],0,1,1.0,D,1.0,NaN,104.8,0.5,0.866025,0.707107,-7.071068e-01,0.201299,0.97953,9.649485,62.350515


In [7]:
all_ref = {}

In [8]:
# pgm ref 생성
pgm_ref = s2p.PGM명.drop_duplicates().reset_index(drop=True)

# 쇼호스트 ref 생성
sh_ref = []
# 없음 추가하기 
for sh_li in s2p.쇼호스트명.drop_duplicates():
  if sh_li is not None:
    for sh in sh_li:
      if sh not in sh_ref:
        sh_ref.append(sh)
  else:
    sh_ref.append('없음')

# 날씨 ref 생성
weather_ref = s2p.날씨.drop_duplicates().reset_index(drop=True)

# 요일 ref 생성
weekday_ref = s2p.weekday.drop_duplicates().reset_index(drop=True)

# 브랜드명 ref 생성
brand_ref = []
for brand_li in s2p.sorted_브랜드명.drop_duplicates():
  if brand_li is not None:
    for brand in brand_li:
      if brand not in brand_ref:
        brand_ref.append(brand)

# 중분류명명 ref 생성
midcat_ref = []
for midcat_li in s2p.comb_상품중분류명.drop_duplicates():
  if midcat_li is not None:
    for midcat in midcat_li:
      if midcat not in midcat_ref:
        midcat_ref.append(midcat)

# 한정표현 ref 생성
expression_ref = []
for brand_li in s2p.한정표현구분.drop_duplicates():
  if brand_li is not None:
    for brand in brand_li:
      if brand not in expression_ref:
        expression_ref.append(brand)

# 가격 ref 생성
price_ref = list(range(0,100000,1))


In [9]:
all_ref['pgm_ref'] = sorted(pgm_ref)
all_ref['showhost_ref'] = sorted(sh_ref)
all_ref['weather_ref'] = sorted(weather_ref)
all_ref['weekday_ref'] = sorted(weekday_ref)
all_ref['brand_ref'] = sorted(brand_ref)
all_ref['midcat_ref'] = sorted(midcat_ref)
all_ref['expression_ref'] = sorted(expression_ref)
all_ref['price_ref'] = sorted(price_ref)

In [10]:
import joblib
joblib.dump(all_ref, 'ref.pkl')

['ref.pkl']

In [12]:
a = pd.DataFrame(columns=['a','b','c'])
a.append({'a':1,'b':2,'c':3},ignore_index=True)

,a,b,c
0,1,2,3


In [17]:
str(datetime.datetime.now())[:-7]

'2022-02-15 23:30:42'

In [19]:
t = joblib.dump(a,'backup2')

In [24]:
a.to_pickle().encode()

TypeError: to_pickle() missing 1 required positional argument: 'path'

In [22]:
def test(df):
  return df.to_pickle().

In [23]:
t = test(a)

TypeError: to_pickle() missing 1 required positional argument: 'path'

In [75]:
a.iloc[1] = [4,5]
a

IndexError: iloc cannot enlarge its target object

In [71]:
pd.DataFrame(columns=['a','b','c'],[1,2,3])

SyntaxError: positional argument follows keyword argument (4192236174.py, line 1)

In [68]:
a = pd.concat([a,pd.Series({'a':1,'b':2,'c':3})],ignore_index=True)

In [69]:
a

,0,a,b,c
0,1.0,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN
4,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN


In [63]:
a.concat({'a':1,'b':2,'c':3},ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'concat'

In [31]:
X_test = pd.read_pickle('/Users/beomso0/Documents/GH/Hyundai/deploy_st/model/X_test(2201)')
y_test = pd.read_pickle('/Users/beomso0/Documents/GH/Hyundai/deploy_st/model/y_test(2201)')

In [30]:
X_test[X_test['pgm_input']==211544]

,date_input,holiday_input,temp_input,pgm_input,start_time_input,end_time_input,duration_input,showhost_input,expression_input,live_input,midecat_input,brand_input,price_input
803,2022-01-01,False,-2,211544,8,11,180.0,"[박민아, 왕영은, 하은지]",[방송에서만],True,"[욕실/청결, 욕실/청결, 주방가전, 건강식품]","[로얄베르겐, 로얄베르겐, 뉴트리닌자, 매일유업]","[89900, 169800, 269000, 189000]"
961,2022-01-08,False,3,211544,8,11,160.0,"[박민아, 왕영은, 하은지]","[단한번, 방송에서만]",True,"[진보석(일반), 가공식품, 가방, 가방, 주방가전, 가방, 진보석(일반)]","[가에따노파리, 맥꾸룸, 핀에스커, 핀에스커, 쿠쿠, 핀에스커, 가에따노파리]","[499000, 119000, 109000, 89000, 458000, 59000,..."
1120,2022-01-15,False,1,211544,8,11,180.0,"[박민아, 왕영은, 하은지]",[방송에서만],True,"[조리용기, 건강식품, 건강식품, 생활가전]","[락앤락, 에이치엘사이언스, 에이치엘사이언스, LG(엘지)]","[178000, 250000, 169000, 1960000]"
1285,2022-01-22,False,3,211544,8,11,161.0,"[박민아, 왕영은, 하은지]",[단한번],True,"[조리용기, 화장품, 계절가전]","[휘슬러, 뷰벨, 케어팟]","[839000, 159000, 159000]"


In [42]:
X_test.iloc[200]

date_input                 2022-01-09 00:00:00
holiday_input                            False
temp_input                                   4
pgm_input                               222934
start_time_input                            12
end_time_input                              13
duration_input                            60.0
showhost_input                           [심용수]
expression_input                       [방송에서만]
live_input                               False
midecat_input                       [컴퓨터, 컴퓨터]
brand_input         [삼성(SAMSUNG), 삼성(SAMSUNG)]
price_input                  [1199000, 999000]
Name: 200, dtype: object

In [43]:
y_test.iloc[200]

160

In [ ]:
'''
0: 4602
1: 225
1: 143
'''

In [26]:
X_train[(X_train['PGM코드']==211544)&(X_train['브랜드1']=='휘슬러')]

KeyError: 'PGM코드'

In [14]:
test01 = test01.sort_values(by='인입콜',ascending= False)

In [17]:
test01

,인입콜,year,쇼호스트명,상품구분,날씨,weekday,duration,판매가평균,price_max_all,price_min_all,prod_num,한정표현_num,주말/공휴일,sh_num,month_sin,month_cos,hour_sin,hour_cos,week,price_max_top3,price_min_top3,brand1,brand2,brand3,mid1,mid2,mid3,price1,price2,price3,score1,score2,score3
961,5877,2,"[박민아, 왕영은, 하은지]",LIVE,흐림,토,160.0,233142,499000,59000,7,2,1,3,0.5,0.866025,0.866025,-0.500000,1,119000,89000,맥꾸룸,핀에스커,핀에스커,가공식품,가방,가방,119000,109000,89000,0.114173,0.085106,0.085106
803,5263,2,"[박민아, 왕영은, 하은지]",LIVE,맑음,토,180.0,179425,269000,89900,4,1,1,3,0.5,0.866025,0.866025,-0.500000,52,269000,89900,로얄베르겐,로얄베르겐,뉴트리닌자,욕실/청결,욕실/청결,주방가전,89900,169800,269000,0.039293,0.039293,0.037490
1120,4961,2,"[박민아, 왕영은, 하은지]",LIVE,구름 많음,토,180.0,639250,1960000,169000,4,1,1,3,0.5,0.866025,0.866025,-0.500000,2,250000,169000,락앤락,에이치엘사이언스,에이치엘사이언스,조리용기,건강식품,건강식품,178000,250000,169000,0.065789,0.033105,0.033105
1285,4602,2,"[박민아, 왕영은, 하은지]",LIVE,구름 많음,토,161.0,385666,839000,159000,3,1,1,3,0.5,0.866025,0.866025,-0.500000,3,839000,159000,휘슬러,뷰벨,케어팟,조리용기,화장품,계절가전,839000,159000,159000,0.065789,0.025162,0.006587
824,4479,2,"[서송이, 심용수]",LIVE,맑음,일,145.0,422333,1199000,39000,6,2,1,2,0.5,0.866025,0.866025,-0.500000,52,129000,39000,USPA,USPA,EXR,스포츠용품,스포츠용품,골프용품,129000,129000,39000,0.047506,0.047506,0.032588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,2,2,없음,DATA,맑음,금,10.0,239000,239000,239000,1,0,0,0,0.5,0.866025,0.500000,0.866025,4,239000,239000,해피한,해피한,해피한,회원권/이용권,회원권/이용권,회원권/이용권,239000,239000,239000,0.004925,0.004925,0.004925
40,1,2,없음,DATA,구름 많음,월,10.0,79900,79900,79900,1,0,0,0,0.5,0.866025,0.500000,0.866025,1,79900,79900,바이오실크,바이오실크,바이오실크,이미용품,이미용품,이미용품,79900,79900,79900,0.045370,0.045370,0.045370
67,1,2,없음,DATA,맑음,화,10.0,79900,79900,79900,1,0,0,0,0.5,0.866025,0.500000,0.866025,1,79900,79900,바이오실크,바이오실크,바이오실크,이미용품,이미용품,이미용품,79900,79900,79900,0.045370,0.045370,0.045370
117,1,2,"[김옥진, 이호준]",DATA,맑음,목,20.0,562000,736000,388000,2,0,0,2,0.5,0.866025,0.707107,0.707107,1,736000,736000,그랜드하얏트,그랜드하얏트,그랜드하얏트,회원권/이용권,회원권/이용권,회원권/이용권,736000,736000,736000,0.004925,0.004925,0.004925
